# load metadata

In [3]:
EGO_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d/'
CAPTION_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d_action_caption'
MOVE_CAPTION_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d_move_action_caption'

import json
import os
import tqdm

from video_caption_action_scene import AnnotationLoader, BetaAlphaCalculator

EGO_VERSION_ROOT = os.path.join(EGO_ROOT, 'v2')
json_path = os.path.join(EGO_ROOT, 'ego4d.json')
train_path = f'{EGO_VERSION_ROOT}/annotations/refined_narration_stream_train.json'
val_path = f'{EGO_VERSION_ROOT}/annotations/refined_narration_stream_val.json'
origin_path = f'{EGO_VERSION_ROOT}/annotations/all_narrations_redacted.json'
video_root = f'{EGO_VERSION_ROOT}/full_scale_2fps'

alpha = 4.9
device = 'cuda:3'
caption_dir = '/root/videollm-online/tmp5'

annotation_loader = AnnotationLoader(train_path, val_path, origin_path, json_path)
narrations = annotation_loader.get_data()
origin_narrations = annotation_loader.get_origin_narration()

beta_alpha_calculator = BetaAlphaCalculator(narrations, alpha)
beta_alpha_calculator.compute_beta()
beta_map = beta_alpha_calculator.get_beta_map()
alpha = beta_alpha_calculator.get_alpha()

import json

main_data = json.load(open("/mnt/extra/dataset/ego4d/v2/annotations/fho_main.json"))
filtered_data = json.load(open("/home/zhangyl/videollm-online/data/estp/ego4d/filtered_data.json"))

In [16]:
print(main_data.keys())

print(main_data['version'])
print(main_data['date'])
print(main_data['description'])
print(main_data['metadata'])
print((main_data['videos'][1].keys()))
print((main_data['videos'][1]['video_uid']))
print((main_data['videos'][1]['video_metadata']))

# structure of a videos
# video : dict
#   - video_uid
#   - video_metadata
#   - annotated_intervals : list[dict]
#       - clip_uid
#       - start_sec
#       - .....(clip metadata)
#       - narrated_actions : list[annotation]
#           - state_transition
#           - time
#           - narration
#           - frame idx
        
# print((main_data['videos'][1]['annotated_intervals'][1]))


print(len(main_data['videos']))
print(len(main_data['videos'][1]['annotated_intervals']))
c = 0
for action in main_data['videos'][100]['annotated_intervals'][1]['narrated_actions']:
    if action['is_valid_action']:
        c+=1
        print(action['narration_text'])
print(c)
print(json.dumps(main_data['videos'][1]['annotated_intervals'][1],indent=4))




dict_keys(['version', 'date', 'description', 'metadata', 'videos'])
2.0
230112
FHO Master Annotation
s3://ego4d-consortium-sharing/public/v2/ego4d.json
dict_keys(['annotated_intervals', 'video_metadata', 'video_uid'])
26202090-684d-4be8-b3cc-de04da827e91
{'video_start_pts': 323, 'video_base_numerator': 1, 'video_base_denominator': 15360, 'duration_sec': 3127.2, 'num_frames': 93816, 'fps': 30.0, 'width': 1440, 'height': 1080, 'rotation': None}
1725
4
#C C picks up an wooden peel board on the wall with his right hand.
#C C holds the wooden peel board with both hands.
#C C puts the wooden peel board on the tray holder of the shelf.
#C C cleans the wooden peel board with his right hand.
#C C flips doughs from the tray on the shelf to the wooden peel board with both hands.
#C C picks up doughs from the tray with both hands.
#C C puts the doughs in his hands on the wooden peel board.
#C C adjusts doughs on the wooden peel board with both hands.
#C C picks up doughs from the tray with both ha

In [4]:

def judge_other(src: str):
    # 1. remove #
    O_list = ['#O', '#o', ' O ', ' o ']
    for o in O_list:
        if o in src:
            return True

soc_annos = {}

for i, anno in enumerate(main_data['videos']):
    
    video_id = anno['video_uid']
    
    if video_id not in filtered_data.keys():
        continue
    
    soc_annos[video_id] = {}
    for clip in anno['annotated_intervals']:
        clip_uid = clip['clip_uid']
        
        soc_annos[video_id][clip_uid] = []
        for action_narration in clip['narrated_actions']:
            # if action_narration['is_valid_action']:
            if judge_other(action_narration['narration_text']):
                print(action_narration['narration_text'])

#O Woman X sits in a room.
#O A woman X holds onto the structure
#O A woman X hands the phone to C
#O Person X walks across the street
#O Man Y rides motorbike
#O Cyclist Z rides a bike
#O The man X adjusts the plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular weight.
#O The man X adjusts the dropped plastic bag to a particular we

In [4]:
print(i)

1724


# state of change task for C

In [5]:
import json
from openai import OpenAI
import os

import random

def get_llm_response_json(system_prompt, user_prompt):

    client = OpenAI(
        api_key="",
        base_url="https://api.deepseek.com",
    )
    
    messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
    )
    
    return response.choices[0].message.content


def clean_text(src: str):
    # 1. remove #
    dst = src.replace('#C', '').replace('#c', '').replace('@c', '').replace('@C', '').replace('C ', '').replace('c ', '')
    dst = dst.replace('#O', '').replace('#o', '').replace('@o', '').replace('@O', '').replace('O ', '').replace('o ', '')
    dst = dst.replace('#Unsure', '').replace('#unsure', '')
    dst = dst.replace('#', '')
    # 2. remove start&end extra space and ,.
    dst = dst.strip('.,\n ') + '.'
    # 3. make the first word capitalize and remove extra space within the sentence
    words = dst.split()
    dst = ' '.join(words)
    
    return dst

system_prompt = open('/home/zhangyl/videollm-online/data/estp/soc/fho_system.txt').read().format(NUMBER=1)
print(system_prompt)
user_prompt_templete = open('/home/zhangyl/videollm-online/data/estp/soc/fho_prompt.txt').read()
output_dir = '/home/zhangyl/videollm-online/dataset/soc_v7/'
os.makedirs(output_dir, exist_ok=True)

soc_annos = {}
for i, anno in enumerate(main_data['videos']):
    if i == 100:
        break
    
    video_id = anno['video_uid']
    
    if video_id not in filtered_data.keys():
        continue
    
    soc_annos[video_id] = {}
    for clip in anno['annotated_intervals']:
        clip_uid = clip['clip_uid']
        
        soc_annos[video_id][clip_uid] = []
        for action_narration in clip['narrated_actions']:
            if action_narration['is_valid_action'] and random.random() < 0.1:
                soc_annos[video_id][clip_uid].append(action_narration)

with open(os.path.join(output_dir, 'soc_select.json'), 'w') as f:
    json.dump(soc_annos, f, indent=4)
    
soc_annos = json.load(open(os.path.join(output_dir, 'soc_select.json')))
c_soc_annos = {}
for k, annos in soc_annos.items():
        
    for clip_id, clip_annos in annos.items():
        os.makedirs(os.path.join(output_dir, k, clip_id), exist_ok=True)
        
        
        for i, action in enumerate(clip_annos):
            
            # HACK: add narration clip---------------
            if k not in c_soc_annos.keys():
                c_soc_annos[k] = {}
            if action['narration_annotation_uid'] not in c_soc_annos[k].keys():
                c_soc_annos[k][action['narration_annotation_uid']] = []
            
            summs = origin_narrations[k]['summaries']
            is_match = False
            for summ in summs:
                if summ['_annotation_uid'] == action['narration_annotation_uid']:
                    is_match = True
                    break
            if not is_match:
                continue
            
            if action['narration_annotation_uid'] not in filtered_data[k].keys():
                continue
            
            clip_start_time = summ['start_time']
            clip_end_time = summ['end_time']
            # HACK: add narration clip---------------
            
            user_prompt = user_prompt_templete.format(clean_text(action['narration_text']))
            response = get_llm_response_json(system_prompt, user_prompt)
            
            # HACK: add format json---------------
            responselist = response.split('\n')
            for res in responselist:
                if res.startswith('**Q:**'):
                    question = res.split('**Q:**')[1].strip()
                elif res.startswith('**A:**'):
                    answer = res.split('**A:**')[1].strip()
            
            qa = {
                'clip_start_time': clip_start_time,
                'clip_end_time': clip_end_time,
                'question': question,
                'Task Type': 'Object State Change Recognition',
                'conversation': [
                    {
                        'role': 'assistant',
                        'content': answer,
                        'time': action['narration_timestamp_sec'],
                        'start_time': action['start_sec'],
                        'end_time': action['end_sec'],
                    }
                ]
            }
            c_soc_annos[k][action['narration_annotation_uid']].append(qa) 
            # HACK: add format json---------------
            
            with open(os.path.join(output_dir, k, clip_id, f'{i}_q.txt'), 'w') as f:
                f.write(user_prompt)
            
            with open(os.path.join(output_dir, k, clip_id, f'{i}_gen.txt'), 'w') as f:
                f.write(response)
        

You are an AI assistant skilled at generating questions and answers. I have an action description involving changes in the state of objects. 

Your task is to:  

1. Extract the state change of the objects involved in the action.  
2. Convert this state change into questions and answers focusing **only on state changes**.  
3. Phrase the questions in a way that indicates a desire for future reminders (e.g., "what is ...", "I want to know when...", "when you see ...").  
4. Provide only 1 best question-answer pairs based on the action description.  
5. Ensure that the questions are varied in structure and rich in content.  
6. Generate questions based on the following question format : 
    - **Q:** Can you remind me of the specific brand written on the cup?
    **A:** The specific brand written on the cup is KFC.
    - **Q:** Is there any text or label on the cup? Could you let me know?
    **A:** Yes, the cup has the text "KFC" on it.
    - **Q:** What brand name is printed on the cup

In [23]:
print(clean_text('#C C flips doughs from the tray on the shelf to the wooden peel board with both hands.'))

C flips doughs from the tray on the shelf to the wooden peel board with both hands.


In [4]:
with open(os.path.join(output_dir, 'c_soc_annos.json'), 'w') as f:
    json.dump(c_soc_annos, f, indent=4)

# state of change for O

In [6]:
import json
from openai import OpenAI
import os

import random

def get_llm_response_json(system_prompt, user_prompt):

    client = OpenAI(
        api_key="",
        base_url="https://api.deepseek.com",
    )
    
    messages = [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
    )
    
    return response.choices[0].message.content


def clean_text(src: str):
    # 1. remove #
    dst = src.replace('#C', '').replace('#c', '').replace('@c', '').replace('@C', '')
    dst = dst.replace('#O', '').replace('#o', '').replace('@o', '').replace('@O', '')
    dst = dst.replace('#Unsure', '').replace('#unsure', '')
    dst = dst.replace('#', '')
    # 2. remove start&end extra space and ,.
    dst = dst.strip('.,\n ') + '.'
    # 3. make the first word capitalize and remove extra space within the sentence
    words = dst.split()
    dst = ' '.join(words)
    
    return dst

system_prompt_judege = open('/home/zhangyl/videollm-online/data/estp/soc/fho_system_judge.txt').read().format(NUMBER=1)
system_prompt = open('/home/zhangyl/videollm-online/data/estp/soc/fho_system.txt').read().format(NUMBER=1)
user_prompt_templete = open('/home/zhangyl/videollm-online/data/estp/soc/fho_prompt.txt').read()
output_dir = '/home/zhangyl/videollm-online/dataset/soc_o_v1/'
os.makedirs(output_dir, exist_ok=True)
soc_annos = {}

for i, anno in enumerate(main_data['videos']):
    if i == 10:
        break
    
    video_id = anno['video_uid']
    
    if video_id not in filtered_data.keys():
        continue
    
    soc_annos[video_id] = {}
    for clip in anno['annotated_intervals']:
        clip_uid = clip['clip_uid']
        
        soc_annos[video_id][clip_uid] = []
        for action_narration in clip['narrated_actions']:
            if judge_other(action_narration['narration_text']):
                user_prompt = user_prompt_templete.format(clean_text(action_narration['narration_text']))
                response = get_llm_response_json(system_prompt_judege, user_prompt)
                
                if 'Yes' in response or 'yes' in response:
                    # print(response)
                    soc_annos[video_id][clip_uid].append(action_narration)
                

with open(os.path.join(output_dir, 'soc_select.json'), 'w') as f:
    json.dump(soc_annos, f, indent=4)
    


In [5]:
soc_annos = json.load(open('/home/zhangyl/videollm-online/dataset/soc_o_v1/soc_select.json'))

In [7]:

output_dir = '/home/zhangyl/videollm-online/dataset/soc_o_v1/'
o_soc_annos = {}

for k, annos in soc_annos.items():  
    for clip_id, clip_annos in annos.items():
        if len(clip_annos) == 0:
            continue
            
        for i, action in enumerate(clip_annos):
            os.makedirs(os.path.join(output_dir, k, clip_id), exist_ok=True)
            
            # HACK: add narration clip---------------
            if k not in o_soc_annos.keys():
                o_soc_annos[k] = {}
            if action['narration_annotation_uid'] not in o_soc_annos[k].keys():
                o_soc_annos[k][action['narration_annotation_uid']] = []
            
            summs = origin_narrations[k]['summaries']
            is_match = False
            for summ in summs:
                if summ['_annotation_uid'] == action['narration_annotation_uid']:
                    is_match = True
                    break
            if not is_match:
                continue
            
            if action['narration_annotation_uid'] not in filtered_data[k].keys():
                continue
            
            clip_start_time = summ['start_time']
            clip_end_time = summ['end_time']
            # HACK: add narration clip---------------
            
            user_prompt = user_prompt_templete.format(clean_text(action['narration_text']))
            response = get_llm_response_json(system_prompt, user_prompt)
            
            # HACK: add format json---------------
            responselist = response.split('\n')
            for res in responselist:
                if res.startswith('**Q:**'):
                    question = res.split('**Q:**')[1].strip()
                elif res.startswith('**A:**'):
                    answer = res.split('**A:**')[1].strip()
            
            qa = {
                'clip_start_time': clip_start_time,
                'clip_end_time': clip_end_time,
                'question': question,
                'Task Type': 'Object State Change Recognition',
                'conversation': [
                    {
                        'role': 'user',
                        'content': answer,
                        'time': action['narration_timestamp_sec'],
                        'start_time': action['start_sec'],
                        'end_time': action['end_sec'],
                    }
                ]
            }
            o_soc_annos[k][action['narration_annotation_uid']].append(qa) 
            # HACK: add format json---------------
            
            
            with open(os.path.join(output_dir, k, clip_id, f'{i}_q.txt'), 'w') as f:
                f.write(user_prompt)
            
            with open(os.path.join(output_dir, k, clip_id, f'{i}_gen.txt'), 'w') as f:
                f.write(response)

with open(os.path.join(output_dir, 'o_soc_annos.json'), 'w') as f:
    json.dump(o_soc_annos, f, indent=4)

In [18]:
file = '/home/zhangyl/videollm-online/data/estp/annotations/o_soc_annos_v2.json'
anno = json.load(open(file))
for k,v in anno.items():
    for kk ,vv in v.items():
        for qa in vv:
            qa['Task Type'] = 'Object State Change Recognition'

json.dump(anno, open(file, 'w'), indent=4)